In [33]:
!pip install onnx onnxruntime

In [1]:
import onnx
import onnxruntime as ort
import numpy as np

# Define input and output tensor names
input_name = "X"
input_axis_name = "A"
unsqueeze_output_name = "Y"

# Create the ONNX model with Unsqueeze operator
def create_unsqueeze_model(axis, input_rank, output_shape, dtype, axis_dtype):

    #Create input tensor
    input_x = onnx.helper.make_tensor_value_info(input_name, dtype, input_rank)
    input_axis = onnx.helper.make_tensor_value_info(input_axis_name, axis_dtype, [len(axis)])

    # Create output tensor (final result after unsqueeze operation)
    unsqueeze_output = onnx.helper.make_tensor_value_info(unsqueeze_output_name, dtype, output_shape)

    # Define unsqueeze node
    unsqueeze_node = onnx.helper.make_node(
    "Unsqueeze",
    inputs=[input_name, input_axis_name],
    outputs=[unsqueeze_output_name],
    )

    # Create the ONNX graph
    graph_def = onnx.helper.make_graph(
        [unsqueeze_node],
        "Unsqueeze",
        [input_x, input_axis],
        [unsqueeze_output],
    )

    # Create the ONNX model
    model = onnx.helper.make_model(graph_def, opset_imports=[onnx.helper.make_opsetid("", 22)])
    model.ir_version = 10 
    onnx.checker.check_model(model)

    # Save the model
    onnx.save(model, "unsqueeze.onnx")

    # Load and run the model with ONNX Runtime
    session = ort.InferenceSession("unsqueeze.onnx")
    return session

def do_unsqueeze(x, axis, session):
    # Run inference
    output = session.run(None, {input_name: x, input_axis_name: axis})

    x_f = (np.array2string(x, separator=',', max_line_width=np.inf).replace('\n', ''))
    y_f = (np.array2string(output[0], separator=',', max_line_width=np.inf).replace('\n', ''))

    # Display results
    print("Shape of input tensor:", x.shape)
    print(f"X={x_f}")
    print("Shape of output tensor:", output[0].shape)
    print(f"Result = {y_f}")


np.set_printoptions(precision=None, floatmode='fixed')

In [2]:
def calculate_output_shape(x, a):
    r = len(x.shape) + len(a)
    a_normalized = []
    for a_value in a:
        if a_value < 0:
            a_normalized.append(a_value + r)
        else:
            a_normalized.append(a_value)

    output_shape = list(x.shape)
    for axis in sorted(a_normalized):
        pos = axis if axis >= 0 else len(output_shape) + axis + 1
        output_shape.insert(pos, 1)
    return output_shape

## Nominal Cases

In [3]:
# Case N1: 3-rank tensor (int32), axes = [0]
onnx_type = onnx.TensorProto.INT32
axes_type = onnx.TensorProto.INT64
x = np.arange(24).reshape(2, 3, 4).astype(np.int32)
axes = np.array([0]).astype(np.int64)  
input_shape = list(x.shape)
output_shape = calculate_output_shape(x, axes)
session = create_unsqueeze_model(axes, input_shape, output_shape, onnx_type, axes_type)
do_unsqueeze(x, axes, session)

Shape of input tensor: (2, 3, 4)
X=[[[ 0, 1, 2, 3],  [ 4, 5, 6, 7],  [ 8, 9,10,11]], [[12,13,14,15],  [16,17,18,19],  [20,21,22,23]]]
Shape of output tensor: (1, 2, 3, 4)
Result = [[[[ 0, 1, 2, 3],   [ 4, 5, 6, 7],   [ 8, 9,10,11]],  [[12,13,14,15],   [16,17,18,19],   [20,21,22,23]]]]


In [4]:
# Case N2: 3-rank tensor (int32), axes = [-1]
onnx_type = onnx.TensorProto.INT32
axes_type = onnx.TensorProto.INT64
x = np.arange(24).reshape(2, 3, 4).astype(np.int32)
axes = np.array([-1]).astype(np.int64)  
input_shape = x.shape
output_shape = calculate_output_shape(x, axes)
session = create_unsqueeze_model(axes, input_shape, output_shape, onnx_type, axes_type)
do_unsqueeze(x, axes, session)

Shape of input tensor: (2, 3, 4)
X=[[[ 0, 1, 2, 3],  [ 4, 5, 6, 7],  [ 8, 9,10,11]], [[12,13,14,15],  [16,17,18,19],  [20,21,22,23]]]
Shape of output tensor: (2, 3, 4, 1)
Result = [[[[ 0],   [ 1],   [ 2],   [ 3]],  [[ 4],   [ 5],   [ 6],   [ 7]],  [[ 8],   [ 9],   [10],   [11]]], [[[12],   [13],   [14],   [15]],  [[16],   [17],   [18],   [19]],  [[20],   [21],   [22],   [23]]]]


In [7]:
# Case N3: 3-rank tensor (int32), axes = [0,1]
onnx_type = onnx.TensorProto.INT32
axes_type = onnx.TensorProto.INT64
x = np.arange(24).reshape(2, 3, 4).astype(np.int32)
axes = np.array([0,1]).astype(np.int64)  
input_shape = x.shape
output_shape = calculate_output_shape(x, axes)
session = create_unsqueeze_model(axes, input_shape, output_shape, onnx_type, axes_type)
do_unsqueeze(x, axes, session)

Shape of input tensor: (2, 3, 4)
X=[[[ 0, 1, 2, 3],  [ 4, 5, 6, 7],  [ 8, 9,10,11]], [[12,13,14,15],  [16,17,18,19],  [20,21,22,23]]]
Shape of output tensor: (1, 1, 2, 3, 4)
Result = [[[[[ 0, 1, 2, 3],    [ 4, 5, 6, 7],    [ 8, 9,10,11]],   [[12,13,14,15],    [16,17,18,19],    [20,21,22,23]]]]]


In [ ]:
# This cell must fail. The axis value is out of range
# Case N4: 3-rank tensor (int32), axes = [4]
onnx_type = onnx.TensorProto.INT32
axes_type = onnx.TensorProto.INT64
x = np.arange(24).reshape(2, 3, 4).astype(np.int32)
axes = np.array([4]).astype(np.int64)  
input_shape = x.shape
output_shape = calculate_output_shape(x, axes)
session = create_unsqueeze_model(axes, input_shape, output_shape, onnx_type, axes_type)
do_unsqueeze(x, axes, session)

2025-12-10 11:57:46.491544509 [E:onnxruntime:, sequential_executor.cc:572 ExecuteKernel] Non-zero status code returned while running Unsqueeze node. Name:'' Status Message: /onnxruntime_src/onnxruntime/core/providers/common.h:31 int64_t onnxruntime::HandleNegativeAxis(int64_t, int64_t) IsAxisInRange(axis, tensor_rank) was false. axis 4 is not in valid range [-4,3]



RuntimeException: [ONNXRuntimeError] : 6 : RUNTIME_EXCEPTION : Non-zero status code returned while running Unsqueeze node. Name:'' Status Message: /onnxruntime_src/onnxruntime/core/providers/common.h:31 int64_t onnxruntime::HandleNegativeAxis(int64_t, int64_t) IsAxisInRange(axis, tensor_rank) was false. axis 4 is not in valid range [-4,3]
